# Importing packages

In [1]:
# !pip install mne

In [2]:
# !pip install pyxdf

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import mne
import pyxdf
import glob
import os

In [ ]:
import warnings
warnings.filterwarnings('ignore') # to ignore warnings

In [4]:
verbose = False                    # global variable to suppress output display of MNE functions
mne.set_log_level(verbose=verbose) # to suppress large info outputs

In [10]:
from xdf import *

# Data Loading

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

In [6]:
magdy_data_path = "magdy\sub-magdy\ses-S001\eeg"
folder_path = magdy_data_path

In [7]:
all_files   = glob.glob(folder_path + '/*eeg.xdf')
len(all_files)     # if  return zero,then no file is loaded

80

# Using our XDF class

In [16]:
from xdf import *

# file = Xdf('BCI Data\sub-Anwar\ses-S001\eeg\sub-Anwar_ses-S001_task-Baseline_run-002_eeg.xdf')
file = Xdf('magdy\sub-magdy\ses-S001\eeg\sub-magdy_ses-S001_task-Right Release_run-003_eeg.xdf')



ValueError: Shape of passed values is (4005, 9), indices imply (4005, 49)

In [ ]:

print(file.getSamplingRate())
file.toCSV('testClass.csv', False)

# from XDF to MNE

In [ ]:
# read stream from xdf
streams, header = pyxdf.load_xdf(all_files[0])
# extract data
data = streams[0]["time_series"].T
# check that it is 9 channels
assert data.shape[0] == 9 # 9 raw EEG channels
#get channels count
ch_count = int(streams[0]["info"]["channel_count"][0])
# extract channels names
ch_names = []
for i in range(ch_count):
  ch_names.append(streams[0]["info"]["desc"][0]["channels"][0]["channel"][i]["label"][0])
# extract sampling ratw
fs = float(streams[0]["info"]["nominal_srate"][0])
# create info
info = mne.create_info(ch_names, fs, "eeg")

In [ ]:
def extract_tasks_names(files):
  task_names = []
  for path in files:
      # Split the path to isolate the filename
      filename = path.split('/')[-1]
      # Extract the task name part
      task_name = filename.split('_')[2]
      # Replace the hyphen with a space to match the desired format
      task_name = task_name.split('-')[-1]
      # Append the task name to the list
      task_names.append(task_name)
  return task_names

In [ ]:
task_names = extract_tasks_names(all_files)
event_id = {'Left Grasp': 1, 'Right Grasp': 2, 'Right Release': 3, 'Baseline': 4}
# Convert the list of task names to the list of numbers
task_numbers = [event_id[task] for task in task_names]
print(task_numbers)

[1, 2, 3, 4, 2, 3, 1, 4, 1, 2, 4, 4, 3, 3, 2, 4, 3, 1, 3, 2, 3, 1, 2, 1, 4, 1, 4, 1, 1, 3, 2, 2, 3, 1, 2, 1, 3, 2, 3, 4, 1, 2, 2, 2, 3, 1, 3, 3, 3, 4, 1, 2, 2, 1, 4, 1, 1, 4, 4, 3, 3, 1, 3, 2, 4, 4, 4, 2, 2, 4, 1, 4, 3, 4, 4, 2, 2, 4, 1, 3]


In [ ]:
# crop parametes
t_start = 1
t_end = 6

idx_start = int(fs*t_start)
idx_end = int(fs*t_end)

In [ ]:
all_data=[]
for fname in all_files:
  streams, header = pyxdf.load_xdf(fname)
  data = streams[0]["time_series"].T
  # crop data from second 1 to 6
  cropped_data = data[:,idx_start:idx_end+1]
  all_data.append(cropped_data)

In [ ]:
# Combine into a 3D array
combined_array = np.stack(all_data, axis=0)
print(combined_array.shape)

In [ ]:
eeg_data = combined_array

In [ ]:
epochs = mne.EpochsArray(eeg_data, info, verbose=verbose, tmin=0)

In [ ]:
# epochs.set_montage('standard_1020')
epochs.filter(1., None)
# epochs.apply_baseline(baseline=(-.250, 0)) # linear baseline correction

epochs.event_id = event_id
epochs.events[:,2] = task_numbers

In [ ]:
epochs

Number of events,80
Events,1: 80
Time range,0.000 – 5.000 s
Baseline,off


In [ ]:
def get_mne_epochs(filepath, verbose=verbose, t_start=1, fs=500, mode='train'):
    '''
    This function reads the EEG data from .xdf file and convert it to MNE-Python Compatible epochs
    data structure. It takes data from [0, 8] sec range and return it by setting t = 0 at cue onset
    i.e. 3 seconds and dropping first two seconds so the output data is in [-1.0, 5.0] sec range. The
    Details can be found in the preprocessing section of the attached document
    '''

    return epochs

def get_labels(filepath):
    mat_data = loadmat(filepath) # read .mat file
    return mat_data['Labels'].ravel()